## from 642 tutorial 3

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

torch.set_printoptions(sci_mode=False, linewidth=300)

In [ ]:
from sklearn.model_selection import train_test_split

X = m2_pipeline
y = m2_pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from torch.utils.data import TensorDataset

X_train = torch.as_tensor(X_train, dtype=torch.float) # an alternative to torch.from_numpy
y_train = torch.as_tensor(y_train, dtype=torch.float)
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class NeuralNetwork_v1(nn.Module):
    def __init__(self):
        super(NeuralNetwork_v1, self).__init__() # initialize the parent class 

        self.fc1   = nn.Linear(in_features=8, out_features=16)  # in_features is the data dim, 8
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Linear(in_features=16, out_features=16) # In middle layers, in_features must match the last out_features
        self.relu2 = nn.ReLU()
        self.fc3   = nn.Linear(in_features=16, out_features=1)  # out_features is the label dim, 1

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_v1 = NeuralNetwork_v1().to(device) # .to method can also move an entire model to device
print(model_v1)

In [ ]:
inputs, labels = next(iter(train_dataloader))
inputs = inputs.to(device) # data must reside on the same device
labels = labels.to(device)

outputs = model_v1(inputs) # call your model as if it were a function
outputs.size()

In [ ]:
x = inputs[:3] #3 data vectors of size 8
print(x.size())

In [ ]:
fc1_out = model_v1.fc1(x) # nn.Linear(in_features=8, out_features=16)
print(fc1_out.size())

In [ ]:
print(f"Before ReLU: {fc1_out}\n\n")
relu1_out = model_v1.relu1(fc1_out) # nn.ReLU()
print(f"After ReLU: {relu1_out}")

In [ ]:
class NeuralNetwork_v2(nn.Module):
    def __init__(self):
        super(NeuralNetwork_v2, self).__init__() # initialize the parent class 

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=8, out_features=16),  # in_features is the data dim, 8
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=16), # In middle layers, in_features must match the last out_features
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=1),  # out_features is the label dim, 1
        )

    def forward(self, x):
        return self.linear_relu_stack(x) # no need to pass x around anymore; modules are called sequentially

In [ ]:
model_v2 = NeuralNetwork_v2().to(device)
outputs = model_v2(inputs) # call your model as if it were a function
outputs.size() # get outputs of the same size as that of model_v1

In [ ]:
class NeuralNetwork_v3(nn.Module):
    def __init__(self):
        super(NeuralNetwork_v3, self).__init__() # initialize the parent class 

        self.fc1 = nn.Linear(in_features=8, out_features=16)  # in_features is the data dim, 8
        self.fc2 = nn.Linear(in_features=16, out_features=16) # In middle layers, in_features must match the last out_features
        self.fc3 = nn.Linear(in_features=16, out_features=1)  # out_features is the label dim, 1

        # homework: use nn.ModuleList to hold the three layers

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(torch.relu(x)) # ReLU now is no longer a "layer", but just a function
        x = self.fc3(torch.relu(x))
        return x

In [ ]:
model_v3 = NeuralNetwork_v3().to(device)
outputs = model_v3(inputs) # call your model as if it were a function
outputs.size() # get outputs of the same size as that of model_v1

In [ ]:
import torch.nn.functional as F

torch.allclose(F.relu(fc1_out), torch.relu(fc1_out))

In [ ]:
print(f"Model structure: {model_v1}\n\n")

for name, param in model_v1.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

predict using an NN: https://stackoverflow.com/a/51058062

resource examples via https://github.com/jcjohnson/pytorch-examples